<a href="https://colab.research.google.com/github/Luaim/Plant-Species-Identification-Model/blob/main/Plant_Species_Identification_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🌱 Plant Identification Model (MobileNetV2, TensorFlow/Keras, TFLite Export)
# Author: Luai - GIthup: Luaim
# Description: Full pipeline - Data, Training, Evaluation, TFLite Export for Flutter app

---


## Step 1: Install & Import Required Libraries

We start by installing and importing all the required Python libraries.  
This includes TensorFlow (for building and training the deep learning model),  
NumPy, Pandas, Matplotlib, and Seaborn (for data manipulation and visualization),  
and scikit-learn (for evaluation).  
We will use MobileNetV2 as our backbone for transfer learning.


In [ ]:
# STEP 1: Install & Import Libraries
!pip install tensorflow seaborn --quiet

import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns


## Step 2: Mount Google Drive and Set Data Paths

We mount Google Drive so we can access our dataset and JSON files stored there.  
Update the path variables (`DATA_DIR` and `JSON_PATH`) to match your folder structure in Drive.


In [ ]:
# STEP 2: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# SET YOUR PATHS HERE:
DATA_DIR = "/content/drive/MyDrive/plantnet_300K/images"
JSON_PATH = "/content/drive/MyDrive/plantnet_300K/plantnet300k_inf.json"


## Step 3: Prepare Label Mapping from JSON

We read the metadata JSON file to create a mapping from image hashes to `species_id` and  
from `species_id` to readable species names.  
This will help us match images to their correct labels and display plant names in the app.


In [ ]:
# STEP 3: Prepare Label Mapping from JSON
with open(JSON_PATH, 'r') as f:
    meta = json.load(f)

image_hash_to_species = {k: v["species_id"] for k,v in meta.items()}
species_id_to_name = {v["species_id"]: v["species_name"] for v in meta.values()}
species_ids = sorted(list(set(image_hash_to_species.values())))
species_id_to_index = {sid: i for i, sid in enumerate(species_ids)}




> ## Step 4: Create Data Generators

We use Keras' `ImageDataGenerator` to load images from folders, automatically labeling them  
based on the folder names (`class_id`).  
The generator also performs real-time data augmentation for the training set to help the model generalize better.




In [ ]:
# STEP 4: Data Generators
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    os.path.join(DATA_DIR, 'train'),
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    os.path.join(DATA_DIR, 'val'),
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_generator = val_datagen.flow_from_directory(
    os.path.join(DATA_DIR, 'test'),
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)


## Step 5: Build the Transfer Learning Model (MobileNetV2)

We create our model using MobileNetV2 as a feature extractor (pretrained on ImageNet).  
We freeze its weights initially and add new layers on top for classifying plant species.  
This approach speeds up training and leverages existing visual knowledge.


In [ ]:
# STEP 5: Build MobileNetV2 Model
base_model = MobileNetV2(
    weights='imagenet',
    include_top=False,
    input_shape=IMG_SIZE + (3,)
)
base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
outputs = Dense(train_generator.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=outputs)

model.compile(
    optimizer=Adam(learning_rate=1e-3),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
model.summary()


## Step 6: Train the Model (Transfer Learning)

We train only the top layers (with the base MobileNetV2 frozen) for several epochs.  
This allows the new layers to learn features specific to your plant dataset.


In [ ]:
# STEP 6: Train Model (initial transfer learning)
EPOCHS = 10
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS
)


## Step 7: Fine-tune the Model

We unfreeze the last few layers of MobileNetV2 and continue training with a lower learning rate.  
This helps the model adapt deeper features to your specific dataset and boosts accuracy.


In [ ]:
# STEP 7: Fine-tune (unfreeze some base layers)
base_model.trainable = True
for layer in base_model.layers[:-50]:
    layer.trainable = False

model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history_finetune = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=5
)


## Step 8: Evaluate and Visualize Results

We plot training and validation accuracy, display a confusion matrix for test predictions,  
and print a classification report to measure the model’s performance.


In [ ]:
# STEP 8: Evaluate and Plot Results
plt.plot(history.history['accuracy'], label='train acc')
plt.plot(history.history['val_accuracy'], label='val acc')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

test_steps = test_generator.samples // BATCH_SIZE + 1
predictions = model.predict(test_generator, steps=test_steps)
y_pred = np.argmax(predictions, axis=1)
y_true = test_generator.classes

cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(12, 10))
sns.heatmap(cm, cmap='Blues', xticklabels=False, yticklabels=False)
plt.title("Confusion Matrix")
plt.show()

print(classification_report(y_true, y_pred))


## Step 9: Export the Model to TensorFlow Lite (TFLite)

We save the trained Keras model, then convert it to TensorFlow Lite format.  
This `.tflite` model can be run efficiently on mobile devices (like in your Flutter app).  
We also export a quantized (smaller/faster) version for mobile deployment.


In [ ]:
# STEP 9: Export to TFLite (for Flutter)
model.save('/content/plantnet_model.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open('/content/plantnet_model.tflite', 'wb') as f:
    f.write(tflite_model)

# (Optional) Quantized model:
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quant_model = converter.convert()
with open('/content/plantnet_model_quant.tflite', 'wb') as f:
    f.write(tflite_quant_model)


## Step 10: Save the Class Index Mapping

We save a JSON file that maps model output indices to the corresponding plant species IDs.  
Your Flutter app will use this to translate model predictions into plant names and rich info.


In [ ]:
# STEP 10: Save class index mapping for app
class_indices = train_generator.class_indices  # class_name -> idx
idx_to_species = {v: k for k, v in class_indices.items()}
with open('/content/species_id_to_index.json', 'w') as f:
    json.dump(idx_to_species, f)


## Step 11: Test Model on a Single Image

We test the trained model on an individual image to verify the prediction,  
and print the predicted species ID and name for manual inspection.


In [ ]:
# STEP 11: Test on Single Image
from tensorflow.keras.preprocessing import image

img_path = '/content/drive/MyDrive/plantnet_300K/images/test/1355868/0a342112ddd74ee3ea7918c445e2133fb5b9454d.jpg'
img = image.load_img(img_path, target_size=IMG_SIZE)
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0) / 255.

pred = model.predict(x)
predicted_idx = np.argmax(pred)
predicted_class = idx_to_species[str(predicted_idx)]
print("Predicted Species ID:", predicted_class)
print("Predicted Species Name:", species_id_to_name[predicted_class])
